
<img align="left" src = https://scienceserver.linea.org.br/images/linea-logo.png width=120 style="padding: 20px"> <br> 
<img align="left" src = https://jupyter.org/assets/homepage/hublogo.svg width=200 style="padding: 20px"> <br> 
<br>
<br>
<br>
<br>

 

Notebook original: Julia Gschwend

Adaptação: Heleno Campos e Marcela Vitti

***


Bem vindo(a) ao LIneA JupyterHub! 

O LIneA JupyterHub oferece acesso a dados públicos disponíveis online e a dados privados de levantamentos fotométricos cuja participação de cientistas brasileiros com _data rights_ é apoiada pelo LIneA. Neste notebook vamos exemplificar o acesso aos dados públicos do levantamento _Gaia_. Caso precise de ajuda, entre em contato pelo e-mail: [helpdesk@linea.org.br](mailto:helpdesk@linea.org.br)


## 1. Sobre os dados


<img align="left" src=https://gaia.aip.de/cms/data/img/eDR3_flux_hammer_1000x500_v1.png width=400 style="background-color:black; padding: 20px; margin-right: 2em"> <br>




O [Observatório Espacial Gaia](https://www.esa.int/Science_Exploration/Space_Science/Gaia) é uma missão da Agência Espacial Europeia (ESA) dedicada a mapear com precisão a posição, a distância e o movimento de mais de 1,8 bilhão de estrelas em toda a Via Láctea, com o objetivo de entender a estrutura e evolução da galáxia. Através de medições de astrometria, fotometria e espectroscopia, o Gaia registra dados em três bandas no espectro visível: G, BP (Blue Photometer) e RP (Red Photometer), permitindo uma caracterização detalhada de estrelas, além da detecção de milhares de novos asteroides, exoplanetas e outros objetos celestes.

O [Gaia Data Release 3 (DR3)](https://www.cosmos.esa.int/web/gaia/data-release-3), disponibilizado ao público em 2022, é o conjunto de dados mais extenso e detalhado da missão até o momento. O DR3 fornece informações astrométricas e fotométricas aprimoradas, além de incluir novos produtos, como composições químicas, temperaturas estelares, velocidades radiais e detecções de variação temporal para cerca de 1,8 bilhão de fontes observadas. Os dados cobrem todo o céu e são acessíveis através de diversos portais, incluindo o Gaia Archive e serviços compatíveis com o protocolo TAP, como o Gaia DR3 TAP+ para acesso via consultas SQL. Ele também pode ser acessado aqui pelo JupyterHub, como veremos a seguir.

## 2. Acesso ao banco de dados 

Dentro da plataforma LIneA JupyterHub, o acesso ao banco de dados é feito através da biblioteca [`dblinea`](https://github.com/linea-it/dblinea), que já está disponível no ambiente. Confira a documentação completa da biblioteca `dblinea` [neste link](https://dblinea.readthedocs.io/en/latest/index.html).


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord

from dblinea import DBBase

%reload_ext autoreload
%autoreload 2

**Leitura dos dados**


A classe `DBBase` faz a conexão com o banco de dados e oferece algumas funcionalidades como veremos a seguir. Nos exemplos abaixo, vamos acessar os dados da tabela **source** do catálogo **gaia_dr3**. 

In [ ]:
db = DBBase()
schema = "gaia_dr3"
tablename = "source"

Para obter a lista de colunas disponíveis: 

In [ ]:
db.get_table_columns(tablename, schema=schema)

Para saber o tipo de dado em cada coluna: 

In [ ]:
db.describe_table(tablename, schema=schema)

A função `fetchall(query)` faz a consulta no banco de dados e retorna uma lista de tuplas com os dados referentes à _query_ fornecida no argumento. Por exemplo, vamos consultar o identificador único e as coordenadas dos objetos nas 10 primeiras linhas da tabela.  

In [ ]:
query = "SELECT source_id, ra, dec FROM gaia_dr3.source limit 10"
lista_10_objetos = db.fetchall(query)
lista_10_objetos

A função `fetchall_dict(query)` faz a consulta no banco de dados e retorna uma lista de dicionários com os dados referentes à _query_ fornecida no argumento. Vamos repetir a consulta do exemplo anterior, ou seja, vamos utilizar o mesmo _SQL statement_ atribuído à variável `query`.    

In [ ]:
dict_10_objetos = db.fetchall_dict(query)
dict_10_objetos

A função `fetchall_df(query)` faz a consulta no banco de dados e retorna um objeto do tipo [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) com os dados referentes à _query_ fornecida no argumento. Vamos repetir a consulta dos exemplo anteriores:

In [ ]:
dataframe_10_objetos = db.fetchall_df(query)
dataframe_10_objetos

### Exemplos 

Para ilustrar a visualização de uma pequena amostra de dados, vamos construir o diagrama cor-magnitude com as estrelas da [galáxia anã de Sculptor](https://en.wikipedia.org/wiki/Sculptor_Dwarf_Galaxy). 

|Coordenadas Equatoriais| |
|:-- |--- | 
|Ascensão Reta| 01h 00m 09.3s |
|Declinação| −33° 42' 33" |


A tabela abaixo traz os significados das colunas que vamos utilizar para fazer a consulta no banco de dados. A lista completa de colunas disponíveis na tabela **source** do **gaia_dr3** está disponível [nesta página](https://userquery.linea.org.br/metadata/gaia_dr3/source/). 


|Coluna | Significado |
|---|---|
|source_id | Unique source identifier (unique within a particular Data Release)|
|ra | Right ascension [degrees]|
|dec | Declination [degrees] |
|phot_g_mean_mag | G-band mean magnitude |
|phot_bp_mean_mag| Integrated BP mean magnitude |
|phot_rp_mean_mag | Integrated RP mean magnitude |

Para fazer uma busca pelas coordenadas no banco, precisamos converter as unidades para graus. Para isto, vamos usar a classe `SkyCoord` do módulo `astropy.coordinates` (veja detalhes na [documentação do Astropy](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html)):

In [ ]:
c = SkyCoord('01h00m09.3s', '−33d42m33s', frame='icrs')
c  

In [ ]:
print(f"R.A.: {c.ra.deg:.1f} degrees")
print(f"Dec.: {c.dec.deg:.1f} degrees")

Os dois exemplos abaixos mostram a consulta das magnitudes nas bandas _g, BP, RP_.

#### **Exemplo 1: seleção de uma região "retangular"**

Para selecionar qualquer amostra com base nas coordenadas de posição, recomenda-se utilizar as funções da biblioteca [Q3C](https://github.com/segasai/q3c) ([Koposov, S., & Bartunov, O. 2006](http://adsabs.harvard.edu/abs/2006ASPC..351..735K)) para tirar vantagem da indexação das colunas. Uma busca por uma região no céu definida por faixas de coordenadas resultaria em uma varredura em todos os ~700 milhões de objetos, o que pode levar um tempo considerável!   


Para consultar objetos dentro de uma região contida em um polígono, basta informar os vértices do polígono pra a função `q3c_poly_query()`. A documentação das funções está disponível no [repositório do Q3C](https://github.com/segasai/q3c).  

Vamos selecionar as magnitudes de uma amostra de estrelas em uma região "quadrada" (na esfera celeste) de lado igual a 1 grau, ou seja, com uma margem de 0.5 graus em torno da posição do nosso alvo. 

**Vértices (ra, dec)**: (14.5, -34.2), (15.5, -34.2), (15.5, -33.2), (14.5, -33.2)

A query ficaria assim (o uso de letras maiúsculas é opcional):  

```sql 
SELECT source_id, ra, dec, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag 
FROM gaia_dr3.source 
WHERE q3c_poly_query(ra, dec, ARRAY[14.5, -34.2, 15.5, -34.2, 15.5, -33.2, 14.5, -33.2])
````

In [ ]:
query_1 = """
    SELECT source_id, ra, dec, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag 
    FROM gaia_dr3.source
    WHERE q3c_poly_query(ra, dec, ARRAY[14.5, -34.2, 15.5, -34.2, 15.5, -33.2, 14.5, -33.2])
"""        
query_1 

In [ ]:
%%time
dados_exemplo_1 = db.fetchall_df(query_1)

In [ ]:
dados_exemplo_1

#### **Exemplo 2: seleção de uma região  circular com Q3C**  

Para selecionar uma região circular, basta informar as coordenadas do centro da seleção e um raio (em graus) nos argumentos da função `q3c_radial_query()`. Para um diâmetro de 1 grau, a query ficaria assim:
    
```sql
    SELECT source_id, ra, dec, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag 
    FROM gaia_dr3.source
    WHERE q3c_radial_query(ra, dec, 15.0, -33.7, 0.5)
````

In [ ]:
query_2 = """
    SELECT source_id, ra, dec, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag 
    FROM gaia_dr3.source
    WHERE q3c_radial_query(ra, dec, 15.0, -33.7, 0.5)
"""
query_2    

In [ ]:
%%time
dados_exemplo_2 = db.fetchall_df(query_2)

In [ ]:
dados_exemplo_2

### Gráficos 

A seguir, veremos exemplos de gráficos estáticos básicos com a biblioteca Matplotlib. 

Documentação da biblioteca: [matplotlib.org](https://matplotlib.org/)

Dicas: [Cheatsheets for Matplotlib users](https://github.com/matplotlib/cheatsheets#cheatsheets)

#### Distribuição espacial

In [ ]:
%%time
plt.figure(figsize=[10,5], dpi=300)
plt.suptitle("Objetos do Gaia DR3", fontsize=14)
plt.subplot(1,2,1)
plt.plot(dados_exemplo_1.ra, dados_exemplo_1.dec, 'k.', alpha=0.1)
plt.xlabel("R.A. (deg)", fontsize=14)
plt.ylabel("Dec. (deg)", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.subplot(1,2,2)
plt.plot(dados_exemplo_2.ra, dados_exemplo_2.dec, 'k.', alpha=0.1)
plt.xlabel("R.A. (deg)", fontsize=14)
plt.ylabel("Dec. (deg)", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

#### Mapa de densidade

In [ ]:
%%time
plt.figure(figsize=[12,5], dpi=300)
plt.suptitle("Objetos do Gaia DR3", fontsize=14)
plt.subplot(1,2,1)
plt.hist2d(dados_exemplo_1.ra, dados_exemplo_1.dec, bins=50)
plt.xlabel("R.A. (deg)", fontsize=14)
plt.ylabel("Dec. (deg)", fontsize=14)
plt.colorbar(label="density of points")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.subplot(1,2,2)
plt.hist2d(dados_exemplo_2.ra, dados_exemplo_2.dec, bins=50)
plt.xlabel("R.A. (deg)", fontsize=14)
plt.ylabel("Dec. (deg)", fontsize=14)
plt.colorbar(label="density of points")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

*** 

Para o gráfico do CMD, vamos utilizar apenas o exemplo 2. 

In [ ]:
dados = dados_exemplo_2
del dados_exemplo_1   # limpando da memória os dataframes
del dados_exemplo_2   # que não vamos mais utilizar 

Por conveniência, podemos alterar os nomes de algumas colunas. 

In [ ]:
new_columns = {"source_id": "object_id",
               "phot_g_mean_mag": "mag_g",
               "phot_bp_mean_mag": "mag_bp",
               "phot_rp_mean_mag": "mag_rp",
              }

dados.rename(columns=new_columns, inplace=True)

In [ ]:
dados.info()

In [ ]:
dados.head()

Cálculo da cor _g-r_ (nova coluna no _dataframe_ **dados**). 

In [ ]:
dados["bp_rp"] = dados.mag_bp - dados.mag_rp

In [ ]:
dados.head()

In [ ]:
dados.count()

In [ ]:
dados = dados.dropna()
dados.count()

In [ ]:
dados.describe()

In [ ]:
plt.figure(figsize=(8, 6))
plt.hexbin(dados.bp_rp, dados.mag_g, gridsize=100, bins='log')
plt.gca().invert_yaxis()  # Inverte o eixo y, pois magnitudes maiores são mais fracas
plt.xlabel('BP - RP (Cor)')
plt.ylabel('G (Magnitude)')
plt.title('Diagrama Cor-Magnitude (CMD) - Gaia DR3')
# plt.xlim(-0.5,1.5)
plt.ylim(22,10)
plt.colorbar(label='Densidade de estrelas')
plt.show()